In [28]:
from sklearn.tree         import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble     import RandomForestClassifier

from sklearn.preprocessing   import LabelEncoder , OneHotEncoder , StandardScaler , MinMaxScaler , Binarizer 
from sklearn.model_selection import train_test_split , GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score , roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_curve , roc_curve

import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as ms
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# datasets

#### 정밀도(Precision)/재현율(recall) trade off
- sklearn Binarizer 클래서 fit_transform() 
- 임계값(Threshold) 보다 크면 1, 작으면 0 반환

In [29]:
x = [[-1, -1, 2], 
    [2, 0, 0], 
    [0, 1.1, 1.2]]

In [30]:
holder = Binarizer(threshold=0.5) # 0.5보다 크면 1, 0.5보다 작으면 0으로 반환
print(holder.fit_transform(x))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 1.]]


In [31]:
titanic = pd.read_csv('./data/titanic_train.csv')

In [32]:
titanic_label = titanic['Survived']
titanic_data  = titanic.drop(['Survived'], axis = 1)

In [33]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코딩 Cabin , Sex , Embarked
def drop_features(frm):
    frm.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return frm


def pre_processing(frm):
    frm['Age'].fillna(frm['Age'].mean(), inplace=True)
    frm['Cabin'].fillna('N', inplace=True)
    frm['Embarked'].fillna('N', inplace=True)
    return frm


def label_encoder(frm):
    frm['Cabin'] = frm['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        encoder = LabelEncoder()
        encoder.fit(frm[feature])
        frm[feature] = encoder.transform(frm[feature])
    return frm

def transform_features(frm):
    frm = drop_features(frm)
    frm = pre_processing(frm)
    frm = label_encoder(frm)
    return frm

In [34]:
titanic_data_feature = transform_features(titanic_data)

In [35]:
def metrics_evaluation(y_test, y_pred):
    print("정확도(Accuracy) : {}\n정밀도(Precision) : {}\n재현율(Recall) : {}\n조화평균(F1) : {}"
         .format(accuracy_score(y_test, y_pred),
                 precision_score(y_test, y_pred),
                 recall_score(y_test, y_pred),
                 f1_score(y_test, y_pred)))

In [37]:
X_train, X_test, y_train, y_test = train_test_split(titanic_data_feature,
                                                    titanic_label,
                                                   test_size = 0.2,
                                                   random_state = 10)

In [38]:
logistic_model = LogisticRegression()
# 학습
logistic_model.fit(X_train, y_train)

LogisticRegression()

In [41]:
# 예측
y_pred = logistic_model.predict(X_test)
print(y_pred)

pandas.core.series.Series